In [4]:
import os
import pandas as pd
import re
from data_cleaning.utils.normalization_utils import (
    clean_destination
)

##  Remark
Destinations are being processed so only official ports are left and nothing else
##

In [5]:

file_path = '../data/1_merged_typed_data.parquet'
output_path = '../data/2_destination_norm.parquet'

if not os.path.exists(file_path):
    print(f"File not found: {file_path}")

df = pd.read_parquet(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1060708 entries, 225260 to 954242
Data columns (total 20 columns):
 #   Column          Non-Null Count    Dtype              
---  ------          --------------    -----              
 0   TripID          1060708 non-null  int64              
 1   StartLatitude   1060708 non-null  float64            
 2   StartLongitude  1060708 non-null  float64            
 3   StartTime       1060708 non-null  datetime64[ns, UTC]
 4   EndLatitude     1060708 non-null  float64            
 5   EndLongitude    1060708 non-null  float64            
 6   EndTime         1060708 non-null  datetime64[ns, UTC]
 7   StartPort       1060708 non-null  string             
 8   EndPort         1060708 non-null  string             
 9   time            1060708 non-null  datetime64[ns, UTC]
 10  shiptype        1060708 non-null  int64              
 11  Length          1060708 non-null  int64              
 12  Breadth         1060708 non-null  int64              
 13

## --- Step 1
Deal with Inconsistent Records (We can have different formats representing the same thing) if they are there.
And convert columns to categorical where appropriate.

In [6]:
def check_unique_values(df):
    """Check unique values in each column of the DataFrame."""
    col_un = {}
    for col in df.columns:
        clean_series = df[col].dropna()
        nunique = clean_series.nunique()
        col_un[col] = nunique
    return col_un

unique_values_before = check_unique_values(df)
unique_values_before

{'TripID': 1126,
 'StartLatitude': 28,
 'StartLongitude': 34,
 'StartTime': 953,
 'EndLatitude': 29,
 'EndLongitude': 47,
 'EndTime': 943,
 'StartPort': 2,
 'EndPort': 2,
 'time': 414193,
 'shiptype': 11,
 'Length': 107,
 'Breadth': 36,
 'Draught': 238,
 'Latitude': 273,
 'Longitude': 1285,
 'SOG': 227,
 'COG': 3602,
 'TH': 361,
 'Destination': 139}

Column: StartPort, Unique values: 2 - seems correct
Column: EndPort, Unique values: 2 - seems correct

Column: Destination, Unique values: 139 - weird

In [7]:
# Case normalization
text_columns = df.select_dtypes(include=['string']).columns
for col in text_columns:
    df[col] = df[col].str.upper()  # Ensure string type and uppercase

unique_values_after = check_unique_values(df)

changed_columns = list(filter(lambda col: unique_values_after[col] != unique_values_before[col], df.columns))
df[changed_columns].dropna().nunique() # Check how many unique values are there after case normalization

Destination    138
dtype: int64

Case normalization has changed the unique values in the following columns: Destination
It seems there are still inconsistencies in the Destination column, so we will need to clean it further.

In [8]:
df['Destination'].unique() #See some examples of the Destination column

<StringArray>
[          'DEHAM', 'DEBRV---->DEHAM',         'HAMBURG',      'DEHAM.ELBE',
  'DEHAM.ELBE.PLT',     'BREMENHAVEN', 'HAMBURG/.AIRBUS',           'STADE',
 'DE.WVN.>.DE.HAM',       'DEHAM.CTA',
 ...
           'HH.PS',          'HH.CTB',        'HALMSTAD',              'HH',
      'FINKENWERD',          'GDANKS',       'NORDENHAM',     'GDANSK...AS',
     'KALININGRAD',    'BRUNSBUETTEL']
Length: 139, dtype: string

In [9]:
# Ensure 'Destination' has at least one alphabetic character and is not just a country code
df['Destination'] = df['Destination'].apply(
    lambda x: pd.NA if not re.search(r'[A-Za-z]', str(x)) or re.match(r'^[A-Z]{2}$', str(x)) else x
)

def find_values_with_special_chars(df):
    """Find values with special characters in the 'Destination' column."""
    return [
        value for value in df['Destination'].unique()
        if re.search(r'[^A-Za-z0-9]', str(value))
    ]


In [10]:
dest_before = find_values_with_special_chars(df)
print(len(dest_before), "unique values before cleaning with special characters")
dest_before

100 unique values before cleaning with special characters


['DEBRV---->DEHAM',
 'DEHAM.ELBE',
 'DEHAM.ELBE.PLT',
 'HAMBURG/.AIRBUS',
 'DE.WVN.>.DE.HAM',
 'DEHAM.CTA',
 'DEBRV>DEHAM',
 'DE.HAM',
 'DEBHV.NOK',
 'DEHAM.EGH',
 'HAMBURG..DE',
 'DEHAM.CTT',
 'DE.BRV>DEHAM',
 'DE.BRV>DE.HAM',
 'HAMBURG/EUR',
 'DEBRV.>.DEHAM',
 'DEHAM.EG',
 <NA>,
 'KLAIPEDA.VIA.NOK',
 'PLGDY.VIA.NOK',
 'GDYNIA.POLAND',
 'GDYNIA.VIA.NOK',
 'GDANSKVIA.NOK',
 'PLGDN.VIA.NOK',
 'DEHAM.>.PLGDY',
 'GDANSK.VIANOK',
 'BLEXEN.ROAD',
 'GDANSK.VIA.NOK',
 'GDYNIA.VIA.NOK.:)',
 'GDYNIA.:)',
 'GDYNIAVIA)NOK',
 'KLJ.VIA.NOK',
 'BE.ANR.>>.PL.GDN',
 'KIEL.CANAL-TRANS',
 'GDYNIA-PLGDY',
 'KLAIPEDA..=SWIN',
 'GDYNIA/N.O.K',
 'PL.GDN',
 'GDANSK....=SWIN',
 'SEAHU.>.DEBRV',
 'DEHAM.>.PLGDY.NOK',
 'DEHAM.>.PL',
 'GDYNIA.VIA.K.CANAL',
 'GDYNIA.VICINOUJSCIE',
 'GDANSK.PILOT',
 'DE.HAM.............',
 'GDYNIA..VIA.NOK',
 'DEBRT.>.DEHAM',
 'DEBRV.EGH',
 'GDANSK.VIA.KIEL.K',
 'GDYNIA...TH',
 'GDANSK.VICEL',
 'GDANSK...!',
 'GDANSK...#INDFARM',
 'GDANSK....KCHIODE',
 'PL.GDY.VIA.NOK',
 'PL.GDY',

## --- Step 2
We can see that we have different formats representing the same thing, like 'HAMBURG' and 'DEHAM' ext.
The data is incredibly messy, we need to handle country codes, special characters, and different formats.

1. Some have country codes
2. Some contain starting port too
3. Some contain type of facilities (e.g., 'ELBE.RC', 'BREMERHAVEN.VIA.NOK')

start > destination
##

In [11]:
# Clean all data
df['Destination'] = df['Destination'].apply(clean_destination)
df['Destination'].unique()

array(['DEHAM', 'DEBRV.>DEHAM', 'HAMBURG', 'DEHAM.ELBE', 'DEHAM.ELBE.PLT',
       'BREMENHAVEN', 'HAMBURG.AIRBUS', 'STADE', 'DE.WVN.>.DE.HAM',
       'DEHAM.CTA', 'DEBRV>DEHAM', 'FINKENWERDER', 'DE.HAM', 'DEBRV',
       'DEBHV.NOK', 'DEHAM.EGH', 'HAMBURG.DE', 'DEHAM.CTT',
       'DE.BRV>DEHAM', 'BREMERHAVEN', 'DEBRE', 'DE.BRV>DE.HAM',
       'HAMBURG.EUR', 'DEBRV.>.DEHAM', 'DEHAM.EG', None,
       'KLAIPEDA.VIA.NOK', 'GDANSK', 'PLGDY.VIA.NOK', 'PLGDY', 'HAMBUG',
       'GDYNIA.POLAND', 'GDYNIA.VIA.NOK', 'GDANSKVIA.NOK', 'GDYNIA',
       'KLAIPEDA', 'PLGDN.VIA.NOK', 'PLGDN', 'DEHAM.>.PLGDY',
       'GDANSK.VIANOK', 'GYDINIA', 'NOK', 'GYDNIA', 'DEBHV',
       'BLEXEN.ROAD', 'GDANSK.VIA.NOK', 'GDYNIAVIANOK', 'KLJ.VIA.NOK',
       'BE.ANR.>.PL.GDN', 'KIEL.CANAL.TRANS', 'GDYNIA.PLGDY',
       'KLAIPEDA.SWIN', 'GDYNIA.N.O.K', 'GDYNI', 'PL.GDN', 'GDANSK.SWIN',
       'SEAHU.>.DEBRV', 'DEHAM.>.PLGDY.NOK', 'DEHAM.>.PL', 'GDYNYA',
       'GDYNIA.VIA.K.CANAL', 'GDYNIA.VICINOUJSCIE', 'GDANSK.PILOT

In [12]:
# Create mask for rows containing '>'
mask = df['Destination'].str.contains('>', na=False)

# Initialize columns (if not already done)
# df['start_fr_dest'] = None
# df['cleaned_destination'] = df['Destination'].copy()
# Split and assign values safely
# df.loc[mask, 'start_fr_dest'] = df.loc[mask, 'Destination'].str.split('>').str[0] #NOTE for now we wont bother

df.loc[mask, 'Destination'] = df.loc[mask, 'Destination'].str.split('>').str[1]
df['Destination'].unique()

array(['DEHAM', 'HAMBURG', 'DEHAM.ELBE', 'DEHAM.ELBE.PLT', 'BREMENHAVEN',
       'HAMBURG.AIRBUS', 'STADE', '.DE.HAM', 'DEHAM.CTA', 'FINKENWERDER',
       'DE.HAM', 'DEBRV', 'DEBHV.NOK', 'DEHAM.EGH', 'HAMBURG.DE',
       'DEHAM.CTT', 'BREMERHAVEN', 'DEBRE', 'HAMBURG.EUR', '.DEHAM',
       'DEHAM.EG', None, 'KLAIPEDA.VIA.NOK', 'GDANSK', 'PLGDY.VIA.NOK',
       'PLGDY', 'HAMBUG', 'GDYNIA.POLAND', 'GDYNIA.VIA.NOK',
       'GDANSKVIA.NOK', 'GDYNIA', 'KLAIPEDA', 'PLGDN.VIA.NOK', 'PLGDN',
       '.PLGDY', 'GDANSK.VIANOK', 'GYDINIA', 'NOK', 'GYDNIA', 'DEBHV',
       'BLEXEN.ROAD', 'GDANSK.VIA.NOK', 'GDYNIAVIANOK', 'KLJ.VIA.NOK',
       '.PL.GDN', 'KIEL.CANAL.TRANS', 'GDYNIA.PLGDY', 'KLAIPEDA.SWIN',
       'GDYNIA.N.O.K', 'GDYNI', 'PL.GDN', 'GDANSK.SWIN', '.DEBRV',
       '.PLGDY.NOK', '.PL', 'GDYNYA', 'GDYNIA.VIA.K.CANAL',
       'GDYNIA.VICINOUJSCIE', 'GDANSK.PILOT', 'DEBRV.EGH',
       'GDANSK.VIA.KIEL.K', 'DEHAMCTA', 'GDYNIA.TH', 'GDANSK.VICEL',
       'GDANSK.INDFARM', 'GDANSK.KCHIODE', '

#### **I CREATED CUSTOM FILE WITH FUNCTION WE WILL USE DOWN**

df_recombined['Destination'].unique()
Now we have somewhat cleaned Destination column, but we still have some inconsistencies. That the same ports have different names.
I didn't find quicker way to do it, rather than manually checking the names and creating a list of names that represent the same port.
We will only a bit automize this process, by using fuzzy matching to find similar names.
And extracting all ports from UpdatedPub150.csv file, which contains ports and their countries.
[link](https://msi.nga.mil/Publications/WPI)

In [13]:
# from fuzzywuzzy import fuzz
#
# def find_fuzzy_matches(destinations, threshold=80, scorer=fuzz.token_set_ratio, show_progress=False):
#     """
#     Find fuzzy matches among destination names.
#
#     Parameters:
#         destinations (list): List of destination strings to compare
#         threshold (int): Minimum similarity score to consider a match (0-100)
#         scorer: Fuzzy matching function (default: token_set_ratio)
#         show_progress (bool): Whether to print progress during processing
#
#     Returns:
#         dict: Dictionary where keys are original names and values are lists of matches
#               with their scores in format [(matched_name, score), ...]
#     """
#     matches = {}
#     total = len(destinations)
#
#     for i, dest in enumerate(destinations, 1):
#         # Skip NAN/empty values
#         if not dest or str(dest).strip().upper() in ('NAN', 'NULL', ''):
#             continue
#
#         if show_progress:
#             print(f"Processing {i}/{total}: {dest[:30]}...", end='\r')
#
#         # Find matches above threshold (excluding self)
#         potential_matches = process.extract(
#             dest,
#             destinations,
#             scorer=scorer,
#             limit=None
#         )
#
#         # Filter matches
#         good_matches = [
#             (match, score)
#             for match, score in potential_matches
#             if score >= threshold and match != dest
#         ]
#
#         if good_matches:
#             matches[dest] = good_matches
#
#     if show_progress:
#         print("\n" + "=" * 50)
#
#     return matches
# def print_fuzzy_matches(matches, min_score=0, group_similar=False):
#     """
#     Print fuzzy matching results in a readable format.
#
#     Parameters:
#         matches (dict): Output from find_fuzzy_matches
#         min_score (int): Minimum score to display
#         group_similar (bool): Whether to group similar matches together
#     """
#     if not matches:
#         print("No matches found")
#         return
#
#     print(f"\nFuzzy matches (score ≥ {min_score}):")
#     print("=" * 60)
#
#     if group_similar:
#         # Group similar matches to avoid duplicates
#         already_matched = set()
#         for dest in sorted(matches.keys()):
#             if dest in already_matched:
#                 continue
#
#             print(f"\nGroup: {dest}")
#             print("-" * 50)
#
#             # Include the original in the group
#             all_in_group = {dest}
#
#             for match, score in matches[dest]:
#                 if score >= min_score:
#                     print(f"  → {match} (score: {score})")
#                     all_in_group.add(match)
#
#                     # Also include matches of matches
#                     if match in matches:
#                         for submatch, subscore in matches[match]:
#                             if subscore >= min_score and submatch not in all_in_group:
#                                 print(f"    → {submatch} (score: {subscore})")
#                                 all_in_group.add(submatch)
#
#             already_matched.update(all_in_group)
#     else:
#         # Simple listing
#         for dest in sorted(matches.keys()):
#             print(f"\n{dest} matches:")
#             print("-" * 50)
#             for match, score in matches[dest]:
#                 if score >= min_score:
#                     print(f"  → {match} (score: {score})")

In [14]:
# unique_dests = df['Destination'].unique().tolist()
#
# # Find matches with threshold of 85
# matches = find_fuzzy_matches(unique_dests, threshold=75, show_progress=True)
#
# # Print results grouped by similarity
# print_fuzzy_matches(matches, min_score=75, group_similar=True)


From this I will manually create a list of names that represent the same port, that do not have 100 match.
As they can be incorrectly matched, and it would be better to do it manually.


In [15]:
from data_cleaning.utils.normalization_utils import match_names

def replace_with_key(df, column):
    df[column] = df[column].apply(lambda x: match_names(x))
    return df

In [ ]:
df = replace_with_key(df, 'Destination')
# df = replace_with_key(df, 'start_fr_dest', full_dict)
df[['Destination']].reset_index().drop_duplicates(subset=['Destination'])

In [ ]:
# df[['start_fr_dest']].reset_index().drop_duplicates(subset=['start_fr_dest'])

In [ ]:
df.drop_duplicates()

In [ ]:
df.to_parquet(output_path)